In [1]:
#BPR
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial 
%matplotlib inline

In [2]:
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
DATA_FOR_BPR = "data_for_BPR.csv"
SAVE_DIR = "../gamedata/train_result/"

In [ ]:
# #data preprocessing
# data = pd.read_csv(DATA_DIR+DATA_FILE)
# gameid = data['gameid'].drop_duplicates()
# gameid_list = []
# gameid_list = list(gameid)
# gameid2newid = {old:new+1 for new, old in enumerate(gameid_list)}
# data['new_game_id'] = data['gameid'].map(gameid2newid)

# events = data['eventname'].drop_duplicates()
# events_list = list(events)
# events2id = {old:new+1 for new,old in enumerate(events_list)}
# data['new_event_id'] = data['eventname'].map(events2id)

In [ ]:
#data.to_csv(DATA_DIR+'data_for_BPR.csv',index=False)

In [ ]:
#data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [ ]:
#data[data['new_game_id']==63]

In [3]:
def load_data(data_path):
    game_events = defaultdict(set)
    game_count = -1
    event_count = -1
    with open(data_path,'r') as f:
        header_line = next(f)
        for line in f.readlines():
            _, _, _, _, _, _, gameid, eventid = line.split(",")
            gameid = int(gameid)
            eventid = int(eventid)
            #print(u,i)
            game_events[gameid].add(eventid)
            game_count = max(gameid, game_count)
            event_count = max(eventid, event_count)
    print ("game_count:", game_count)
    print ("event_count:", event_count)
    #return max_u_id, max_i_id, user_ratings
    return game_count,event_count,game_events
    

#data_path = os.path.join('D:\\tmp\\ml-100k', 'u.data')
#user_count, item_count, user_ratings = load_data(data_path)
game_count,event_count,game_events_dict = load_data(DATA_DIR+DATA_FOR_BPR)

game_count: 1108
event_count: 11189


In [4]:
#
def generate_test(game_events_dict):
    user_test = dict()
    for u, i_list in game_events_dict.items():
        user_test[u] = random.sample(game_events_dict[u], 1)[0]
    return user_test

game_test_set = generate_test(game_events_dict)

In [9]:
#generate dataset for training
def generate_train_batch(game_events_dict, game_test_set,game_count, event_count, batch_size=512):
    t = []
    
    #print('generate batch..')
    for b in range(batch_size):
        break_flag1 = 0
        break_flag2 = 0
        u = random.sample(game_events_dict.keys(), 1)[0]
        #print('u:',u)
        i = random.sample(game_events_dict[u], 1)[0]
        while i == game_test_set[u]:
            break_flag1 += 1
            i = random.sample(game_events_dict[u], 1)[0]
            if break_flag1 == 20:
                game_events_dict.pop('u',None)
                game_count -= 1
                game_test_set.pop('u',None)
                break
            #print('i:',i)
        j = random.randint(1, event_count)
        while j in game_events_dict[u]:
            break_flag2 += 1
            j = random.randint(1, event_count)
            if break_flag2 == 20:
                game_events_dict.pop('u',None)
                game_count -= 1
                game_test_set.pop('u',None)
                break
            #print('j:',j)
        t.append([u, i, j])
        #print(u,i,j)
    #print('generate batch done!')
    return game_count, numpy.asarray(t)

#train_set = generate_train_batch(game_events_dict,game_test_set,event_count)

In [10]:
def generate_test_batch(game_events_dict, game_test_set, event_count):
    for u in game_events_dict.keys():
        t = []
        i = game_test_set[u]
        for j in range(1, event_count+1):
            if not (j in game_events_dict[u]):
                t.append([u, i, j])
        yield numpy.asarray(t)

In [11]:
def bpr_mf(game_count, event_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [game_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [event_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    # MF predict: u_i > u_j
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keepdims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

In [32]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(game_count, event_count, 20)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    print('start epoch')
    for epoch in range(1, 51):
        print ("epoch: ", epoch)
        _batch_bprloss = 0
        for k in range(1, 5000): # uniform samples from training set
            game_count,uij = generate_train_batch(game_events_dict, game_test_set,game_count,event_count)

            _bprloss, _train_op = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")

        game_count = 0
        _auc_sum = 0.0

        # each batch will return only one user's auc
        for t_uij in generate_test_batch(game_events_dict, game_test_set, event_count):

            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                    feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]}
                                )
            game_count += 1
            _auc_sum += _auc
        print ("test_loss: ", _test_bprloss, "test_auc: ", _auc_sum/game_count)
        print ("")
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    saver.save(session, SAVE_DIR)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)
print('all done')

start epoch
epoch:  1
bpr_loss:  0.7225075693673242
_train_op
test_loss:  1.274829 test_auc:  0.5016026170507711

epoch:  2
bpr_loss:  0.7208527729472629
_train_op
test_loss:  1.2732158 test_auc:  0.5018094794574973

epoch:  3
bpr_loss:  0.7198388016707994
_train_op
test_loss:  1.2715724 test_auc:  0.5019019438959436

epoch:  4
bpr_loss:  0.7191005266769144
_train_op
test_loss:  1.2698967 test_auc:  0.5020976291868238

epoch:  5
bpr_loss:  0.7184701009639909
_train_op
test_loss:  1.2682152 test_auc:  0.5024190555619885

epoch:  6
bpr_loss:  0.7179820251288379
_train_op
test_loss:  1.2665474 test_auc:  0.5028054839788011

epoch:  7
bpr_loss:  0.7174491928228977
_train_op
test_loss:  1.2649364 test_auc:  0.5033355412443378

epoch:  8
bpr_loss:  0.7170180671929025
_train_op
test_loss:  1.2633543 test_auc:  0.5039359121425275

epoch:  9
bpr_loss:  0.716582158120257
_train_op
test_loss:  1.2617518 test_auc:  0.5046266028690652

epoch:  10
bpr_loss:  0.7161377805498844
_train_op
test_loss:  

In [ ]:
saver.save(session, SAVE_DIR)

In [15]:
session1 = tf.Session()
u1_dim = tf.expand_dims(values[0][0], 0)
u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)
result_1 = session1.run(u1_all)
print (result_1)

[[-0.03434188  0.03079957  0.00685023 ...  0.01158312  0.05417788
  -0.06018631]]


In [31]:
print("new recommedation ：")
p = numpy.squeeze(result_1)
p[numpy.argsort(p)[:-5]] = 0
for index in range(len(p)):
    if p[index] != 0:
        select = data[data['new_event_id']==index].drop_duplicates('eventname')
  
        print (index, p[index],select['eventname'])

new recommedation ：
2187 0.12627804 197127    ftueFreeBellsAcknowledged1
Name: eventname, dtype: object
4073 0.12869741 356524    cardPlayed
Name: eventname, dtype: object
10564 0.15509363 823075    missionReward
Name: eventname, dtype: object
10632 0.14570697 826063    mowStart
Name: eventname, dtype: object
11118 0.13249832 866053    upgradeCoachBuilding
Name: eventname, dtype: object


In [17]:
data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [28]:
data[data['new_event_id']==11118]
data[data['new_game_id']==1].drop_duplicates('eventname')

,gameid,genre,platform,eventname,eventparameter,eventtype,new_game_id,new_event_id
0,3,No genre,ios,transaction,focus,custom,1,1
1,3,No genre,ios,gameEnded,attack,custom,1,2
2,3,No genre,ios,missionFailed,attack,custom,1,3
3,3,No genre,ios,newPlayer,attack,standard,1,4
4,3,No genre,ios,levelUp,attack,custom,1,5
5,3,No genre,ios,characterCreated,attack,custom,1,6
6,3,No genre,ios,guild,attack,custom,1,7
7,3,No genre,ios,social,attack,custom,1,8
9,3,No genre,ios,clientDevice,attack,standard,1,9
10,3,No genre,ios,missionCompleted,attack,custom,1,10
